In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("Open-Orca/OpenOrca")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from nltk.tokenize import word_tokenize

import pandas as pd

In [5]:
filtered_dataset = dataset['train'].filter(lambda example: len(word_tokenize(example['response'])) >= 100)

Filter:   0%|          | 0/4233923 [00:00<?, ? examples/s]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(filtered_dataset['response'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

duplicates = set()
for i in range(len(cosine_sim)):
    for j in range(i+1, len(cosine_sim)):
        if cosine_sim[i][j] > 0.95:
            duplicates.add(j)

dataset_cleaned = dataset.drop(list(duplicates))

In [ ]:
from datasets import load_dataset
from transformers import LlamaForMaskedLM, LlamaTokenizer, Trainer, TrainingArguments
import torch

In [ ]:
model_name = "allenai/llama-large-1B"
model = LlamaForMaskedLM.from_pretrained(model_name)
tokenizer = LlamaTokenizer.from_pretrained(model_name)

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    num_train_epochs=5,
    logging_dir='./logs',
)

In [ ]:


# Tokenize the cleaned dataset
tokenized_dataset = tokenizer(dataset_cleaned['response'].tolist(),
                              padding=True,
                              truncation=True,
                              return_tensors="pt")

# Define a function to preprocess the dataset
def preprocess_dataset(dataset):
    return {"input_ids": dataset["input_ids"], "labels": dataset["input_ids"]}

# Preprocess the dataset
dataset = tokenized_dataset.map(preprocess_dataset)

# Define the Trainer for fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine-tuned-llama2")


